In [10]:
import tensorflow as tf
import keras
from keras.layers import Dense, Dropout, Input
from keras.models import Sequential

In [51]:
import os

VIRTUAL_HEIGHT = 6 # os.environ["VIRTUAL_HEIGHT"]
VIRTUAL_WIDTH = 6 # os.environ["VIRTUAL_WIDTH"]
DIM = VIRTUAL_WIDTH * VIRTUAL_HEIGHT
BATCH_SIZE = 16
MOVE_CODE = {0:"right", 1:"left", 2:"up", 3:"down"}

## Data stuff

In [12]:
from glob import glob
from objects import Table, Food, Snake
import pickle as pkl
def load_dataset(folder_path="dataset", maximum=None):
    paths = glob(folder_path + "/*")

    games = []
    for path in paths[:maximum if maximum else len(paths)]:
        with open(path, "rb") as fin:
            table = pkl.load(fin)
            games.append(table)
            
    return games

init_games = load_dataset(maximum=None)

In [14]:
len(init_games)

100

## Model

In [29]:
import random
import numpy as np
class TrainFeeder:
    def __init__(self, init_games, n=BATCH_SIZE):
        assert len(init_games) >= n
        self.init_games = init_games
        self.games = random.choices(self.init_games, k=n)
    
    def feed(self):
        vectors = [game.table.ravel() for game in self.games]
        for vector in vectors:
            yield vector
            
    def apply_moves(self, directions: list):
        results = []
        for idx, direction in enumerate(directions):
            result = self.games[idx].snake.move(direction, return_state=True)
            results.append(result)
            
            if result == -1:
                # Reset the game
                self.games[idx] = random.choice(self.init_games)
            
        return np.array(results)
    
feeder = TrainFeeder(init_games=init_games, n=2)

In [44]:
feeder.games

[array([[0., 0., 0., 0., 0., 0.],
        [1., 1., 0., 2., 0., 0.],
        [0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0.]]),
 array([[0., 0., 0., 0., 0., 0.],
        [0., 0., 1., 1., 0., 0.],
        [0., 2., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0.]])]

In [43]:
feeder.apply_moves(["right", "right"])

You lost
You lost


array([-1, -1])

In [ ]:
def loss_fct(model, y_true=None, y_pred):
    global feeder
    
    moves = [MOVE_CODE[code] for code in np.argmax(y_pred, axis=1)]
    y_pred = feeder.apply_moves(moves)
    
    # Change the values inside y_pred because
    # it contains 1 if it is success and -1 if failure
    # and we want to minimize the loss
    y_pred = - y_pred
    
    return tf.sum(y_pred)

In [11]:
model = Sequential()
model.add(Input((DIM, )))
model.add(Dense(64))
model.add(Dense(4, activation="softmax")) #

In [23]:
model.compile(optimizer="adam", loss=loss_fct)
model.summary()

NameError: name 'model' is not defined

In [ ]:
# Add callbacks

model.fit(x=feeder.feed,
    epochs=18,
    verbose=1,
    callbacks=None,
    validation_split=0.2
)